In [1]:
import json
import logging
from collections import OrderedDict
from typing import Optional

import coloredlogs
from ph4_walkingpad.profile import Profile, calories_walk2_minute, calories_rmrcb_minute
from ph4_walkingpad.analysis import StatsAnalysis

logger = logging.getLogger(__name__)
coloredlogs.CHROOT_FILES = []
coloredlogs.install(level=logging.INFO)


In [2]:
an = StatsAnalysis()
an.profile_file = '/Users/dusanklinec/workspace/ph4-walkingpad/profile.json'
an.stats_file = '/Users/dusanklinec/walking.json'
an.load_profile()
an.load_stats()

for ix, m in enumerate(an.loaded_margins):
            print('============================================= Margins: %s, segms: %s' % (ix,len(m)))
            print(json.dumps(m, indent=2))


============================================= Margins: 0, segms: 1
[
  {
    "time": 0,
    "dist": 0,
    "steps": 0,
    "speed": 0,
    "app_speed": 0,
    "belt_state": 0,
    "controller_button": 0,
    "manual_mode": 1,
    "raw": "f8a200000100000000000000000000000000a3fd",
    "rec_time": 1615647793.792786,
    "pid": "ph4r05",
    "ccal": null,
    "ccal_net": null,
    "ccal_sum": null,
    "ccal_net_sum": null,
    "_breaking": true
  }
]
============================================= Margins: 1, segms: 1
[
  {
    "time": 0,
    "dist": 0,
    "steps": 6148,
    "speed": 3,
    "app_speed": 0,
    "belt_state": 0,
    "controller_button": 0,
    "manual_mode": 1,
    "raw": "f8a200030100000000000000180400000000c2fd",
    "rec_time": 1615647793.039586,
    "pid": "ph4r05",
    "ccal": null,
    "ccal_net": null,
    "ccal_sum": null,
    "ccal_net_sum": null,
    "_breaking": true
  }
]
============================================= Margins: 2, segms: 37
[
  {
    "time": 3368,

In [5]:
res = an.load_last_stats()
print(res)

2021-03-13 19:25:03 phx.local ph4_walkingpad.analysis[57039] INFO Calories burned so far this walk:    0.00 kcal,    0.00 kcal net


([], [])
